In [3]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
df_all_zones = pd.read_csv('data/taxi+_zone_lookup.csv')

In [6]:
df_all = pd.read_csv('processed_data/data_taxis_nyc_2018.csv', parse_dates=['tpep_pickup_datetime'])

In [7]:
df_all = df_all.drop(columns=['Unnamed: 0'])

In [8]:
df = df_all.copy()

In [9]:
df_weather = pd.read_csv('data/NY_Boroughs_Weathers_01-2018.csv', parse_dates=['Datetime'])

Agregando al info del clima al df principal.

In [10]:
bors = df_all_zones.Borough.unique()
df_bors = pd.DataFrame(enumerate(bors), columns=['IdBorough', 'Borough'])
df_weather = df_weather.merge(right=df_bors, how='inner', on='Borough')
df_weather['Key_1'] = df_weather.Datetime.dt.strftime('%Y%m%d%H')
df_weather.IdBorough = [*map(str, df_weather.IdBorough)]
df_weather['Key_final'] = df_weather.IdBorough + df_weather.Key_1

In [11]:
df = df.merge(right=df_all_zones, how='left', left_on='PULocationID', right_on='LocationID')
df = df.merge(right=df_bors, how='inner', on='Borough')
df['Key_1'] = df['tpep_pickup_datetime'].dt.strftime('%Y%m%d%H')
df.IdBorough = [*map(str, df.IdBorough)]
df['Key_final'] = df.IdBorough + df.Key_1
df = df.merge(right=df_weather, how='left', on='Key_final')

In [12]:
columns = list(df_all) + ['Temperature', 'Precip_Type']
df = df[columns]

# Creación de tablas

**Vendor**

In [14]:
df_vendor = pd.DataFrame({
    'VendorID': [1,2],
    'Vendor': ['Creative Mobile Technologies, LLC', 'VeriFone Inc.']
})

**Borough**

In [15]:
boroughs = df_all_zones['Borough'].unique()

In [16]:
df_borough = pd.DataFrame({
    'Borough': boroughs
})
df_borough.insert(loc=0, column='BoroughID', value=range(1, len(df_borough)+1))

**Zone**

In [17]:
df_zone = df_all_zones[['LocationID', 'Zone', 'Borough', 'service_zone']]
df_zone.loc[df_zone.LocationID==265, 'Zone'] = 'Unknown'
df_zone.rename(columns={
    'LocationID': 'ZoneID'
}, inplace=True)

In [18]:
def replaceForIndex( p_x ):
    for i, elem in enumerate(df_borough['Borough']):
        if elem == p_x:
            return df_borough.iloc[i]['BoroughID']

In [19]:
df_zone['BoroughID'] = df_zone['Borough'].apply(replaceForIndex)

In [20]:
def replaceForIndex( p_x ):
    for i, elem in enumerate(df_service_zone['Service_Zone']):
        if elem == p_x:
            return df_service_zone.iloc[i]['Service_ZoneID']

In [24]:
df_zone['Service_ZoneID'] = df_zone['service_zone'].apply(replaceForIndex)

KeyError: 'service_zone'

In [22]:
df_zone = df_zone.drop(columns=['Borough', 'service_zone'])

**Service_Zone**

In [23]:
service_zones = df_all_zones['service_zone'].unique()

In [25]:
df_service_zone = pd.DataFrame({
    'Service_Zone': service_zones
})
df_service_zone.insert(loc=0, column='Service_ZoneID', value=range(1, len(df_service_zone)+1))
df_service_zone.loc[df_service_zone['Service_Zone'].isna(), 'Service_Zone'] = 'Unknown'

**Rate_Code**

In [26]:
df_rate_code = pd.DataFrame({
    'RateCodeID': [1,2,3,4,5,6],
    'RateCode':['Standard rate', 'JFK', 'Newark', 'Nassau or Westchester', 'Negotiated fare', 'Group ride']
})

**Payment_Type**

In [27]:
df_payment_type = pd.DataFrame({
    'PaymentTypeID': [1,2,3,4,5,6],
    'PaymentType':['Credit card', 'Cash', 'No charge', 'Dispute', 'Unknown', 'Voided trip']
})

**Precip_Type**

In [28]:
df_precip_type = pd.DataFrame({
    'Precip_Type': ['No precipitacion', 'Rain', 'Snow', 'Rain and Snow']
})
df_precip_type.insert(loc=0, column='Precip_TypeID', value=range(len(df_precip_type)))

**Calendar**

In [30]:
df['tpep_pickup_datetime'] = [*map(str, df['tpep_pickup_datetime'])]
only_dates = df['tpep_pickup_datetime'].str.split()

for i, elem in enumerate(only_dates):
    only_dates[i] = elem[0]
    
unique_dates = list(only_dates.unique())

In [31]:
df_calendar = pd.DataFrame({
    'Date': unique_dates
})
df_calendar.insert(loc=0, column='DateID', value=range(1, len(df_calendar)+1))

df_calendar['Date'] = pd.to_datetime(df_calendar['Date'])

In [32]:
df_calendar['Year']     = df_calendar['Date'].dt.year
df_calendar['Month']    = df_calendar['Date'].dt.month
df_calendar['Day']      = df_calendar['Date'].dt.day
df_calendar['Week']     = df_calendar['Date'].dt.week
df_calendar['Day_of_Week'] = df_calendar['Date'].dt.day_of_week

C:\Users\Dell\AppData\Local\Temp\ipykernel_4076\840377510.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_calendar['Week']     = df_calendar['Date'].dt.week


**Trip**

In [33]:
datetime = df['tpep_pickup_datetime'].str.split()
dates = []
times = []
for date, time in datetime:
    dates.append(date)
    times.append(time)

In [34]:
df_trip = pd.DataFrame({
    'VendorID': df['VendorID'],
    'Date': dates,
    'PU_Time': times,
    'Duration': df['Travel_time'],
    'Passenger_count': df['passenger_count'],
    'Distance': df['trip_distance'],
    'PU_IdZone': df['PULocationID'],
    'DO_IdZone': df['DOLocationID'], 
    'Temperature': df['Temperature'],
    'Precip_Type': df['Precip_Type']
})

In [35]:
df_trip.insert(loc=0, column='IdTrip', value=range(len(df_trip)))

In [36]:
def addBorough( p_x ):
    for i, elem in enumerate(df_zone['ZoneID']):
        if elem == p_x:
            return df_zone.iloc[i]['Borough']

In [37]:
df_trip['Borough'] = df_trip['PU_IdZone'].apply(addBorough)

KeyError: 'Borough'

**Payment**

In [38]:
df_payment = df[['RatecodeID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'improvement_surcharge', 'tolls_amount', 'total_amount']]

In [39]:
df_payment.insert(loc=0, column='IdTrip', value=df_trip['IdTrip'])

In [40]:
df_payment.insert(loc=0, column='IdPayment', value=range(len(df_payment)))

### Renombrar Columnas - Normalizar

In [41]:
df_vendor.columns = ['IdVendor', 'Vendor']
df_calendar.rename(columns={'DateID':'IdDate'}, inplace=True)
df_borough.columns = ['IdBorough', 'Borough']
df_service_zone.rename(columns={'Service_ZoneID':'IdService_Zone'}, inplace=True)
df_trip.rename(columns={'VendorID': 'IdVendor'}, inplace=True)
df_precip_type.rename(columns={'Precip_TypeID': 'IdPrecip_Type'}, inplace=True)
df_rate_code.rename(columns={'RateCodeID': 'IdRate_Code', 'RateCode': 'Rate_Code'}, inplace=True)
df_payment_type.rename(columns={'PaymentTypeID': 'IdPayment_Type','PaymentType': 'Payment_Type'}, inplace=True)
df_zone.rename(columns={'ZoneID': 'IdZone','BoroughID': 'IdBorough','Service_ZoneID': 'IdService_Zone','service_zone': 'Service_Zone'},inplace=True)
df_payment.rename(columns={'RatecodeID': 'IdRate_Code','payment_type': 'IdPayment_Type','fare_amount': 'Fare_Amount','extra': 'Extra','mta_tax': 'MTA_Tax','improvement_surcharge': 'Improvement_Surcharge','tolls_amount': 'Tolls_Amount','total_amount': 'Total_Amount'}, inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_4076\1564989441.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_payment.rename(columns={'RatecodeID': 'IdRate_Code','payment_type': 'IdPayment_Type','fare_amount': 'Fare_Amount','extra': 'Extra','mta_tax': 'MTA_Tax','improvement_surcharge': 'Improvement_Surcharge','tolls_amount': 'Tolls_Amount','total_amount': 'Total_Amount'}, inplace=True)


### Exportar a archivos CSV

In [ ]:
df_vendor.to_csv('tables/vendor.csv')
df_calendar.to_csv('tables/calendar.csv')
df_borough.to_csv('tables/borough.csv')
df_service_zone.to_csv('tables/service_zone.csv')
df_trip.to_csv('tables/trip.csv')
df_precip_type.to_csv('tables/precip_type.csv')
df_rate_code.to_csv('tables/rate_code.csv')
df_payment_type.to_csv('tables/payment_type.csv')
df_zone.to_csv('tables/zone.csv')
df_payment.to_csv('tables/payment.csv')